<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/master/recipes/ocr_pre_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import OCR pre-annotations

In this tutorial we will see how to import OCR pre-annotations in Kili using [Google vision API](https://cloud.google.com/vision/docs/ocr).

Pre-annotating your data will allow you to gain a significant time when performing OCR using Kili. 

The data we use comes from [The Street View Text Dataset](http://www.iapr-tc11.org/mediawiki/index.php?title=The_Street_View_Text_Dataset).

## Loading an image from The Street View Dataset in Kili

You can obtain the image for this tutorial on this [link](https://drive.google.com/uc?export=view&id=1ceNwCgLwIyyjPwU42xIoz6mMT3enLewW):

<img src="https://drive.google.com/uc?export=view&id=1ceNwCgLwIyyjPwU42xIoz6mMT3enLewW" width="800">

We will use the Google to perform an optical caracter recognition of the different texts in the image.


To use the google API, we need to install some packages:

In [ ]:
!pip install google-cloud google-cloud-vision Pillow kili google-cloud-storage wget

In [ ]:
import os
import io
import json

from google.cloud import vision
from google.oauth2 import service_account
from kili.client import Kili
from PIL import Image
import wget
import getpass

We can now create the interface we will be using in our project.

For OCR, the interface to use is a classification job with nested transcriptions for each category:

In [ ]:
json_interface = {
    "jobs": {
        "JOB_0": {  # this job is for annotating the bounding boxes
            "mlTask": "OBJECT_DETECTION",
            "tools": ["rectangle"],
            "instruction": "Categories",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {
                    "STORE_INFORMATIONS": {"name": "Store informations", "children": ["JOB_1"]},
                    "PRODUCTS": {"name": "Products", "children": ["JOB_2"]},
                },
                "input": "radio",
            },
        },
        "JOB_1": {
            "mlTask": "TRANSCRIPTION",
            "instruction": "Transcription of store informations",
            "required": 1,
            "isChild": True,
        },
        "JOB_2": {
            "mlTask": "TRANSCRIPTION",
            "instruction": "Transcription of products",
            "required": 1,
            "isChild": True,
        },
    }
}

Let's authenticate to Kili and create our project:

In [ ]:
if "KILI_API_KEY" not in os.environ:
    KILI_API_KEY = getpass.getpass("Please enter your API key: ")
else:
    KILI_API_KEY = os.environ["KILI_API_KEY"]

kili = Kili(api_key=KILI_API_KEY)

# Create an OCR project
project = kili.create_project(
    description="OCR street view",
    input_type="IMAGE",
    json_interface=json_interface,
    title="Street text annotation",
)
project_id = project["id"]

## Creating OCR annotations using Google Vision API

We will now see how to perform OCR on our image using Google Vision API.

First you will need to create an account on https://cloud.google.com:

  - [create a project](https://console.cloud.google.com/projectcreate) (or use an existing one)
  - then go to the [Cloud Vision API page](https://console.cloud.google.com/apis/library/vision.googleapis.com)
  - activate the API for your project (you might need to associate facturation information if you haven't already)
  
Now that the API is activated we will need to get an API in order to call later in our project:

  - go to [API and services page](https://console.cloud.google.com/apis/credentials)
  - and [create a service account](https://console.cloud.google.com/iam-admin/serviceaccounts/create) with authorization to use the vision API
  
On the [service account details](https://console.cloud.google.com/iam-admin/serviceaccounts) page:

  - click on add a key
  - download the key using json format
  - place the key in your environment variables or in a file

In [ ]:
google_key_str = os.getenv("KILI_API_CLOUD_VISION")

if not google_key_str:
    path_to_json_key = "./google_cloud_key.json"
    with open(path_to_json_key) as file:
        google_key_str = file.read()

GOOGLE_KEY = json.loads(google_key_str)

assert GOOGLE_KEY

We can now start to add OCR annotations to the asset metadata:

In [ ]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [ ]:
def detect_text(path):
    """Detects text in the file."""
    credentials = service_account.Credentials.from_service_account_info(GOOGLE_KEY)
    client = vision.ImageAnnotatorClient(credentials=credentials)

    with io.open(path, "rb") as image_file:
        content = image_file.read()

    response = client.text_detection({"content": content})
    texts = response._pb.text_annotations
    text_annotations = []

    for text in texts:
        vertices = [{"x": vertex.x, "y": vertex.y} for vertex in text.bounding_poly.vertices]

        tmp = {
            "description": text.description,
            "boundingPoly": {
                "vertices": vertices,
            },
        }

        text_annotations.append(tmp)

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )

    return text_annotations

In [ ]:
PATH_TO_IMG = wget.download(
    "https://raw.githubusercontent.com/kili-technology/kili-python-sdk/master/recipes/img/store_front.jpeg"
)

In [ ]:
text_annotations = detect_text(PATH_TO_IMG)
assert len(text_annotations) > 0
print(f"Found {len(text_annotations)} boxes of text.")
print(text_annotations[0])

Found 22 boxes of text.
{'description': "CD\nITALIAN $1\nESPRESSO Shot\nplus tax\nFINE ITALIAN\nIMPORTS & DELI\nJIM\nIMMY'S FRESH MEATS\nSAUSAGES\nFOOD STORE\nIX", 'boundingPoly': {'vertices': [{'x': 24, 'y': 6}, {'x': 1668, 'y': 6}, {'x': 1668, 'y': 553}, {'x': 24, 'y': 553}]}}


In [ ]:
im = Image.open(PATH_TO_IMG)
IMG_WIDTH, IMG_HEIGHT = im.size
print(im.size)

(1680, 1050)


We now need to convert the OCR predictions to the Kili asset metadata format:

In [ ]:
full_text_annotations = {
    "fullTextAnnotation": {
        "pages": [{"height": IMG_HEIGHT, "width": IMG_WIDTH}],
    },
    "textAnnotations": text_annotations,
}

We follow the Google Vision API [`AnnotateImageResponse`](https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse) format. So in the end, the OCR data to insert into Kili as a JSON metadata contains:

- [Full text annotation](https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse#TextAnnotation). A list of pages in the document with their respective heights and widths.
- [A list of text annotations](https://cloud.google.com/vision/docs/reference/rest/v1/AnnotateImageResponse#EntityAnnotation) with:
    - the text content;
    - coordinates of the bounding box.

```json
{
  "fullTextAnnotation": { "pages": [{ "height": 914, "width": 813 }] },
  "textAnnotations": [
    {
      "description": "7SB75",
      "boundingPoly": {
        "vertices": [
          { "x": 536, "y": 259 },
          { "x": 529, "y": 514 },
          { "x": 449, "y": 512 },
          { "x": 456, "y": 257 }
        ]
      }
    },
    {
      "description": "09TGG",
      "boundingPoly": {
        "vertices": [
          { "x": 436, "y": 256 },
          { "x": 435, "y": 515 },
          { "x": 360, "y": 515 },
          { "x": 361, "y": 256 }
        ]
      }
    }
  ]
}
```

Let's upload the asset with its pre-annotations to Kili:

In [ ]:
external_id = "store"
content = PATH_TO_IMG

kili.append_many_to_dataset(
    project_id=project_id,
    content_array=[content],
    external_id_array=[external_id],
    json_metadata_array=[full_text_annotations],
)

In [ ]:
assets = kili.assets(project_id=project_id)
assert len(assets) > 0

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.94it/s]


## Annotate in Kili

You can now annotate your images and you will see the text automatically extracted:

<img src="https://raw.githubusercontent.com/kili-technology/kili-python-sdk/master/recipes/img/store_with_ocr.gif" width="800">

Congrats! 👏

Pre-annotating your assets can save you a significant amount of time and improve the accuracy of your labeling ⏳🎯.

## Cleanup

To clean up, we simply need to remove the project that we created:

In [ ]:
kili.delete_project(project_id)